In [1]:
# %pip install cuda
%pip install cupy

Defaulting to user installation because normal site-packages is not writeable
  Using cached cupy-10.5.0.tar.gz (1.7 MB)
  Created wheel for cupy: filename=cupy-10.5.0-cp39-cp39-win_amd64.whl size=62096440 sha256=7972e60d4143402ef04368ad06ecabcce57662ce41b0d0c44281906f9faadc6a
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\af\7b\67\b2e056d901a3b1a5b6320cf628349e5e0690e5a54b300f6155
Successfully built cupy
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np

In [2]:
# import numpy as np
from scipy.stats import poisson, norm, multivariate_normal
from numpy.linalg import inv
from numpy.linalg import multi_dot

# from cupy.linalg import inv
# from cupy.linalg import multi_dot

# General funtions

In [3]:
def pois_ll(X, y, beta):
    Xb = np.dot(X,beta)
    ll = np.dot(Xb,y) - np.sum(np.exp(Xb))
    return ll

def pois_KL(X, y, beta, theta = None):
    Xb = np.dot(X,beta)
    if theta is None:
        rel = y
    else:
        rel = theta
    kl_1 = np.dot(np.exp(rel),rel-Xb)
    kl_2 = np.exp(rel) - np.exp(Xb)
    KL = np.sum(kl_1-kl_2)
    return KL

def nb_ll(X, y, beta, alpha):
    Xb = np.dot(X,beta)
    ll = np.dot(y,Xb) - np.dot(np.ones(y.shape[0]) * alpha + y , np.log(np.exp(Xb) + alpha))
    return np.sum(ll)

def nb_KL(X, y, beta, alpha ,theta = None):
    Xb = np.dot(X,beta)
    if theta is None:
        rel = y
    else:
        rel = theta
    kl_1_par1 = np.log(np.exp(rel) / (np.exp(rel) + alpha))
    kl_1_par2 = np.log(np.exp(Xb) / (np.exp(Xb) + alpha))
    kl_1 = np.dot(exp(rel),kl_1_par1-kl_1_par2)
    
    kl_2 = np.log((np.exp(Xb) + alpha) / (np.exp(y) + alpha)) * alpha
    
    KL = np.sum(kl_1+kl_2)
    return KL

# IRLS and Forward Selection

In [4]:
########################
#### IRLS Algorithm ####
########################

def IRLS(X, y, reg_type: ['poisson','nb']
         , alpha = None, threshold = 0.01
         , just_score = True):    
    beta = np.zeros((X.shape[1]))
    #### Distribution specifics ####
    ## Poisson    
    if reg_type == 'poisson':
        ll_cur = pois_ll(X, y, beta)
        W = np.diagflat(np.exp(np.dot(X,beta)))
        D = np.diagflat(np.exp(np.dot(X,beta)))
        z = np.dot(X,beta) + np.dot(inv(D),(y-np.exp(np.dot(X,beta))))
        beta_est = multi_dot([inv(multi_dot([np.transpose(X),W,X])) , np.transpose(X), W, z])
        ll_next = pois_ll(X, y, beta_est)
        
        # IRLS part
        
        while ll_next - ll_cur > threshold:
            ll_cur = pois_ll(X, y, beta_est)
            W = np.diagflat(np.exp(np.dot(X,beta_est)))
            D = np.diagflat(np.exp(np.dot(X,beta_est)))
            z = np.dot(X,beta_est) + np.dot(inv(D),(y-np.exp(np.dot(X,beta_est))))
            beta_est = multi_dot([inv(multi_dot([np.transpose(X),W,X])) , np.transpose(X), W, z])
            ll_next = pois_ll(X, y, beta_est)
    ## NB        
    if reg_type == 'nb':
        ll_cur = nb_ll(X, y, beta, alpha)
        W = np.diagflat(np.exp(np.dot(X,beta))/(1+(alpha**-1) * np.exp(np.dot(X,beta)))) # adjust to nb
        D = np.diagflat(np.exp(np.dot(X,beta)))
        z = np.dot(X,beta) + np.dot(inv(D),(y-np.exp(np.dot(X,beta))))
        beta_est = np.dot(multi_dot([inv(multi_dot([np.transpose(X),W,X])) , np.transpose(X), W]), z)
        ll_next = nb_ll(X, y, beta_est, alpha)
        
        # IRLS part
        
        while ll_next - ll_cur > threshold:
            ll_cur = nb_ll(X, y, beta_est, alpha)
            W = np.diagflat(np.exp(np.dot(X,beta_est))/(1+(alpha**-1) * np.exp(np.dot(X,beta_est)))) # adjust to nb
            D = np.diagflat((np.exp(np.dot(X,beta_est))))
            z = np.dot(X,beta_est) + np.dot(inv(D),(y-np.exp(np.dot(X,beta_est))))
            beta_est = np.dot(multi_dot([inv(multi_dot([np.transpose(X),W,X])) , np.transpose(X), W]), z)
            ll_next = nb_ll(X, y, beta_est, alpha)
    
    if just_score:
        return ll_next
    else:
        return beta_est, ll_next, np.exp(np.dot(X,beta_est))

###########################
#### Forward Algorithm ####
###########################

def fwd(X, y, 
        reg_type: ['poisson','nb'], criteria: ['AIC','BIC','RIC']
        , sel_feat = None, sel_dict = None, alpha = None):
    """
    X, y, 
        reg_type: ['poisson','nb'], criteria: ['AIC','BIC','RIC']
        , sel_feat = None, sel_dict = None, alpha = None
    """
    if sel_feat is None:
        sel_feat = []
    if sel_dict is None:
        sel_dict = {'features':[],'score': np.inf}
#         feat_score =
    for feature in [i for i in range(X.shape[1]) if i not in sel_feat]:
#         print(sel_feat)
#         print(feature)
        in_feat = sel_feat+[feature]
#         print(in_feat)
#         print('currently testing:')
#         print(in_feat)
        # Criteria definition:
        if criteria == 'AIC':
            pen = len(in_feat)
        if criteria == 'BIC':
            pen = 0.5 * np.log(X.shape[0])* len(in_feat)
        if criteria == 'RIC':
            pen = np.log(X.shape[1])* len(in_feat)
        if criteria == 'NLP': #NLP - Non-Linear penalty
            pen = len(in_feat) * np.log(X.shape[1] * np.exp(1) / len(in_feat))
        feat_score_next = -IRLS(X[:,in_feat],y,reg_type,alpha)
        if feat_score_next < sel_dict['score']:
            feat_score = feat_score_next + pen # taking the ll score
            sel_dict['features'] = in_feat
            sel_dict['score'] = feat_score
            sel_dict['-ll']  = feat_score_next
    return sel_dict

# FISTA

In [5]:
from numpy import linalg as LA

def pois_nll_grad(X,y,beta):
    Xb = np.dot(X,beta).astype(np.float64)
#     exp_ob = 
    nll_grad = np.dot(np.transpose(X),np.exp(Xb) - y)
    return nll_grad

def nb_nll_grad(X,y,beta,alpha):
    Xb = np.dot(X,beta).astype(np.float64)
    sec_factor = (np.exp(Xb) - y)/(np.exp(Xb) + alpha)
    nll_grad = alpha * np.dot(np.transpose(X),sec_factor)
    return nll_grad

def L_pois(X,y):
    eigs = LA.eigh(np.dot(np.transpose(X),X))[0]
    idx = eigs.argsort()[::-1][0]  
    eig_max = eigs[idx]
    return np.mean(y) * eig_max
    
def L_nb(X,y,alpha):
    eigs = LA.eigh(np.dot(np.transpose(X),X))[0]
    idx = eigs.argsort()[::-1][0]  
    eig_max = eigs[idx]
    return (alpha + np.mean(y))/alpha * eig_max

def prox(grad, beta, L, pen_vec):
    prox_inp = beta - 1/L * grad
    prox_out = np.maximum(np.abs(prox_inp) - pen_vec,0) * np.sign(prox_inp)
    return prox_out

def FISTA(X, y, pen_vec, 
          type: ['poisson', 'nb'],
          iterations = 1000, is_ordered = True
          ,alpha = None
          ):
    """
    X - Design matrix
    y - Dependent variables
    pen_vec - The penalty vector
    type - The regression type - Poisson or NB
    Iterations - Number of iterations 
    is_ordered - True
    alpha - Only relevant if we use NB
    """
    beta_start = np.zeros(X.shape[1], dtype = np.float64)
    w_start = np.zeros(X.shape[1], dtype = np.float64)
    delta_start = 1
    for k in range(iterations):
        if type == 'poisson':
            grad = pois_nll_grad(X, y, beta_start)
            L = L_pois(X,y)
        elif type == 'nb':
            grad = nb_nll_grad(X, y, beta_start,alpha)
            L = L_nb(X,y,alpha)
        ### Ordering for the thresholding ###
        
        if is_ordered:

            indx = np.argsort(beta_start)[::-1]

            beta_start = beta_start[indx]
            ind_dict = {i: j for i,j in zip([*range(beta_start.shape[0])], indx)}
        ### Starting the FISTA. Pay attention that we must sort grad according to indx
        
        w_next = prox(grad[indx], beta_start, L, pen_vec)
        delta_next = (1 + np.sqrt(1 + 4*delta_start**2))/2
        beta_next = w_next + ((delta_start - 1)/delta_next)*(w_next - w_start)
        
        ### Re-ordering again for calculating the gradient
        
        beta_start = np.zeros(beta_next.shape[0])
        for origin_ind in ind_dict:
            beta_start[origin_ind] = beta_next[ind_dict[origin_ind]]
        delta_start = delta_next
        w_start = w_next
    
#         if k % 100 == 0:
#             print(k)
#             print('beta_start: {}'.format(beta_start))
#             print('beta_next: {}'.format(beta_next))
#             if type == 'poisson':
#                 print('nll :{}'.format(-pois_ll(X,y,beta_next)))
#             elif type == 'nb':
#                 print('nll :{}'.format(-nb_ll(X,y,beta_next)))
    return beta_next, ind_dict

# Simulations

## Simulations Set

In [3]:
import numpy as np
rho = [0,0.5,0.8]
n = 200
d = [20, 100, 200, 500, 1000]
per = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.5, 0.7, 0.9]
simulation_settings = {i: [] for i in d}

def d_0(d, per, n):
    d_0 = per*np.minimum(d,n)
    return int(d_0)

for di in d:
#     print('d: {}'.format(di))
    for peri in per:
#         print('per: {}'.format(peri))
#         print(d_0(di,peri,n))
        simulation_settings[di].append(d_0(di,peri,n))
# d_0(di,peri,n)
simulation_settings

{20: [1, 2, 3, 4, 5, 6, 7, 10, 14, 18],
 100: [5, 10, 15, 20, 25, 30, 35, 50, 70, 90],
 200: [10, 20, 30, 40, 50, 60, 70, 100, 140, 180],
 500: [10, 20, 30, 40, 50, 60, 70, 100, 140, 180],
 1000: [10, 20, 30, 40, 50, 60, 70, 100, 140, 180]}

In [4]:
simulation_settings

{20: [1, 2, 3, 4, 5, 6, 7, 10, 14, 18],
 100: [5, 10, 15, 20, 25, 30, 35, 50, 70, 90],
 200: [10, 20, 30, 40, 50, 60, 70, 100, 140, 180],
 500: [10, 20, 30, 40, 50, 60, 70, 100, 140, 180],
 1000: [10, 20, 30, 40, 50, 60, 70, 100, 140, 180]}

In [10]:
counter = 0
score_start = np.inf
final_scores = {'AIC': {}, 'BIC': {} ,'RIC': {}}
for crit in ['AIC','BIC','RIC']:
    print('using penalty:{}'.format(crit))
#     criteria = crit
    score_start = np.inf
    sel_dict = fwd(X[:300,:],y[:300],'poisson', crit)
#     print(sel_dict)
    counter = 0
    while score_start > sel_dict['score']:
        counter +=1
        score_start = sel_dict['score']
        sel_dict = fwd(X[:300,:],y[:300],'poisson',crit, sel_feat = sel_dict['features'], sel_dict = sel_dict)
#         print('finished')
#         print(counter)
#     print('Results')
#     print(sel_dict)
    final_scores[crit] = sel_dict

using penalty:AIC
currently testing:
[0]
currently testing:
[1]
currently testing:
[2]
currently testing:
[3]
currently testing:
[4]
currently testing:
[5]
currently testing:
[6]
currently testing:
[7]
currently testing:
[8]
currently testing:
[9]
currently testing:
[10]
currently testing:
[11]
currently testing:
[12]
currently testing:
[13]
currently testing:
[14]
currently testing:
[15]
currently testing:
[16]
currently testing:
[17]
currently testing:
[18]
currently testing:
[19]
currently testing:
[20]
currently testing:
[21]
currently testing:
[22]
currently testing:
[23]
currently testing:
[24]
currently testing:
[25]
currently testing:
[26]
currently testing:
[27]
currently testing:
[28]
currently testing:
[29]
currently testing:
[30]
currently testing:
[31]
currently testing:
[32]
currently testing:
[33]
currently testing:
[34]
currently testing:
[35]
currently testing:
[36]
currently testing:
[37]
currently testing:
[38]
currently testing:
[39]
currently testing:
[40]
currentl

In [11]:
# final_scores.items()
sorted(final_scores.items(), key=lambda item: item[1]['score'])

[('AIC',
  {'features': [49, 48],
   'score': 303.6656246224452,
   '-ll': 299.6656246224452}),
 ('BIC',
  {'features': [49, 48],
   'score': 311.0731895717576,
   '-ll': 299.6656246224452}),
 ('RIC',
  {'features': [49, 48],
   'score': 315.31371664415775,
   '-ll': 299.6656246224452})]

## Simluations Functions

In [7]:
from numpy.linalg import norm
from numpy.random import multivariate_normal
import multiprocessing
import itertools

##### What is left?
#### 1. KL - Over y test and Theta
#### 2. Model Size

def runner(X, y, theta
           , reg_type: ['poisson','nb']
#            , model_type: ['fwd','LASSO','SLOPE']
           , pen_coef = np.exp(np.arange(-40,40,0.3))):
    
    # Creating train and test sets
    d = X.shape[1]
    (X_train, y_train, theta_train), (X_test, y_test, theta_test) = train_test_allocator(X, y, theta)
    print(X_train.shape)
    print(y_train.shape)
    
    model_scores_dict = {'FWD':{'nll':0,'KL':0, 'KL_theta_train':0, 'KL_theta_test':0, 'size':0}
                         ,'LASSO':{'nll':0,'KL':0, 'KL_theta_train':0, 'KL_theta_test':0, 'size':0}
                         ,'SLOPE':{'nll':0,'KL':0, 'KL_theta_train':0, 'KL_theta_test':0, 'size':0}}
    # Creating k-folds #
    indices = np.array([*range(X_train.shape[0])])
    k_folds_inds = np.split(indices,5)
    
    #####################
    # Forward selection #
    #####################
    print('Starting Forward Selection')
    
#     final_scores = {'AIC': {}, 'BIC': {} ,'RIC': {}}
    folds_score = {'AIC': 0, 'BIC': 0 ,'RIC': 0, 'NLP':0}
    # K-fold part

    for crit in ['AIC','BIC','RIC','NLP']:
        print('using penalty:{}'.format(crit))
        score_start = np.inf
#         folds_score = {'AIC': 0, 'BIC': 0 ,'RIC': 0}
        for oos_fold in k_folds_inds:
            fit_folds = [i for i in indices if i not in oos_fold]
            # val set
            val_set = X_train[oos_fold,:]
            val_y = y_train[oos_fold]
            # fit set
            fit_set = X_train[fit_folds,:]        
            fit_y = y_train[fit_folds]
            sel_dict = fwd(X = fit_set,y = fit_y,reg_type = reg_type, criteria = crit)
            while score_start > sel_dict['score']:
#                 counter +=1
                score_start = sel_dict['score']
                sel_dict = fwd(fit_set,fit_y, reg_type,crit
                               , sel_feat = sel_dict['features'], sel_dict = sel_dict)
            
            selected_features = sel_dict['features']
            beta_est_vals = IRLS(fit_set[:,selected_features], fit_y, reg_type, just_score = False)[0]
            beta_est = np.zeros(d)
            beta_est[selected_features] = beta_est_vals
            print(beta_est)
            if reg_type == 'poisson':
                ll = pois_ll(val_set, val_y, beta_est)
            elif reg_type == 'nb':
                ll = nb_ll(val_set, val_y
                             , beta_est, alpha)
            folds_score[crit] += ll
            print(folds_score)
        folds_score[crit] = folds_score[crit]/5
#         final_scores[crit] = folds_score[crit]
    
    # Taking the best option:
    
    print(sorted(folds_score.items(), key=lambda item: item))
    best_score_crit = sorted(folds_score.items(), key=lambda item: item[1])[0][0]
    print('cv selection: {}'.format(best_score_crit))
    # Fitting over entire train set:
    
    score_start = np.inf
    sel_dict = fwd(X_train, y_train, reg_type, best_score_crit)
#     counter = 0
    while score_start > sel_dict['score']:
#         counter +=1
        score_start = sel_dict['score']
        sel_dict = fwd(X_train, y_train, reg_type, best_score_crit
                       , sel_feat = sel_dict['features'], sel_dict = sel_dict)
    final_feautre_set = sel_dict['features']
    
    final_beta_vals = IRLS(X_train[:,final_feautre_set], y_train, reg_type, just_score = False)[0]
    final_beta_est = np.zeros(d)
    final_beta_est[final_feautre_set] = final_beta_vals
    # Testing over the test set:
    
    if reg_type == 'poisson':
        nll = -pois_ll( X_test, y_test, final_beta_est)
        KL = pois_KL( X_test, y_test, final_beta_est, theta = None)
        KL_theta_train = pois_KL( X_train, y_train, final_beta_est, theta = theta_train)
        KL_theta_test = pois_KL( X_test, y_test, final_beta_est, theta = theta_test)
    elif reg_type == 'nb':
        nll = -nb_ll( X_test, y_test
                     , final_beta_est, alpha)
        KL = nb_KL( X_test, y_test
                     , final_beta_est, alpha, theta = None)
        KL_theta_train = nb_KL( X_train, y_train
                     , final_beta_est, alpha, theta = theta_train)
        KL_theta_test = nb_KL( X_test, y_test
                     , final_beta_est, alpha, theta = theta_test)
    
    model_scores_dict['FWD']['nll'] += nll
    model_scores_dict['FWD']['KL'] += KL
    model_scores_dict['FWD']['KL_theta_train'] += KL_theta_train
    model_scores_dict['FWD']['KL_theta_test'] += KL_theta_test
    model_scores_dict['FWD']['size'] += len(final_feautre_set)
   
    #####################
    ### LASSO & SLOPE ###
    #####################
    
    print("Starting LASSO & SLOPE")
    
#     d = X.shape[1]
    pen_vec_LASSO = np.ones(d) * np.sqrt(2 * np.log(d))
    print(pen_vec_LASSO)
    pen_vec_SLOPE = np.array([np.sqrt(np.log(2*d/(j+1))) for j in range(d)])
    print(pen_vec_SLOPE)
    
    
    pen_coef_results = {'SLOPE':{},'LASSO':{}}
    for C in pen_coef:
        SLOPE_cv_score = 0
        LASSO_cv_score = 0
        print('testing C: {}'.format(C))
        for oos_fold in k_folds_inds:
            fit_folds = [i for i in indices if i not in oos_fold]
            # val set
            val_set = X_train[oos_fold,:]
            val_y = y_train[oos_fold]
            # fit set
            fit_set = X_train[fit_folds,:]        
            fit_y = y_train[fit_folds]
            if reg_type == 'poisson':
                # SLOPE
                SLOPE_cv_beta = FISTA(fit_set,fit_y,C*pen_vec_SLOPE,reg_type)[0]
                SLOPE_cv_score += -pois_ll(val_set, val_y, SLOPE_cv_beta)
                
                # LASSO
                LASSO_cv_beta = FISTA(fit_set,fit_y,C*pen_vec_LASSO,reg_type)[0]
                LASSO_cv_score += -pois_ll(val_set, val_y, LASSO_cv_beta)
                
            elif reg_type == 'nb':
                # SLOPE
                SLOPE_cv_beta = FISTA(fit_set,fit_y,C*pen_vec_SLOPE,reg_type, alpha = alpha)[0]
                SLOPE_cv_score += -nb_ll(val_set, val_y, SLOPE_cv_beta, alpha = alpha)
                # LASSO
                LASSO_cv_beta = FISTA(fit_set,fit_y,C*pen_vec_LASSO,reg_type, alpha = alpha)[0]
                LASSO_cv_score += -nb_ll(val_set, val_y, LASSO_cv_beta, alpha = alpha)
        
        pen_coef_results['SLOPE'][C] = SLOPE_cv_score/5
        pen_coef_results['LASSO'][C] = LASSO_cv_score/5
    
    # Finding the best penalty value
    
    # Eliminating nans #
    
    pen_coef_results['SLOPE'] = {pen: score for pen, score in pen_coef_results['SLOPE'].items() if np.isnan(score) == False}
    pen_coef_results['LASSO'] = {pen: score for pen, score in pen_coef_results['LASSO'].items() if np.isnan(score) == False}
        
    print('SLOPE result')
    print(pen_coef_results['SLOPE'].items())
    print({key: val for key, val in sorted(pen_coef_results['SLOPE'].items(), key=lambda item: item[1])})
    print('LASSO result')
    print(pen_coef_results['LASSO'].items())
    print({key: val for key, val in sorted(pen_coef_results['LASSO'].items(), key=lambda item: item[1])})
    SLOPE_sel_pen = sorted(pen_coef_results['SLOPE'].items(), key=lambda item: item[1])[0][0]
    LASSO_sel_pen = sorted(pen_coef_results['LASSO'].items(), key=lambda item: item[1])[0][0]
    
    
    # Fitting and testing over the best penalty value:
    if reg_type == 'poisson':
#         Fit
        SLOPE_final_beta = FISTA(fit_set,fit_y,SLOPE_sel_pen*pen_vec_SLOPE,reg_type)[0]
        SLOPE_final_beta_size = len(SLOPE_final_beta[SLOPE_final_beta > 0])
        
        LASSO_final_beta = FISTA(fit_set,fit_y,LASSO_sel_pen*pen_vec_LASSO,reg_type)[0]
        LASSO_final_beta_size = len(LASSO_final_beta[LASSO_final_beta > 0])
        
#        eval SLOPE
        SLOPE_nll = -pois_ll( X_test, y_test, SLOPE_final_beta)
        SLOPE_KL = pois_KL( X_test, y_test, SLOPE_final_beta, theta = None)
        SLOPE_KL_theta_train = pois_KL( X_train, y_train, SLOPE_final_beta, theta = theta_train)
        SLOPE_KL_theta_test = pois_KL( X_test, y_test, SLOPE_final_beta, theta = theta_test)
        
#         eval LASSO
        LASSO_nll = -pois_ll( X_test, y_test, LASSO_final_beta)
        LASSO_KL = pois_KL( X_test, y_test, LASSO_final_beta, theta = None)
        LASSO_KL_theta_train = pois_KL(X_train, y_train, LASSO_final_beta, theta = theta_train)
        LASSO_KL_theta_test = pois_KL(X_test, y_test, LASSO_final_beta, theta = theta_test)
    
    elif reg_type == 'nb':
#         Fit
        SLOPE_final_beta = FISTA(fit_set,fit_y,SLOPE_sel_pen*pen_vec_SLOPE,reg_type, alpha = alpha)[0]
        LASSO_final_beta = FISTA(fit_set,fit_y,LASSO_sel_pen*pen_vec_LASSO,reg_type, alpha = alpha)[0]
#        eval
        #        eval SLOPE
        SLOPE_nll = -nb_ll( X_test, y_test, SLOPE_final_beta)
        SLOPE_KL = nb_KL( X_test, y_test, SLOPE_final_beta, theta = None)
        SLOPE_KL_theta_train = nb_KL( X_train, y_train, SLOPE_final_beta, theta = theta_train)
        SLOPE_KL_theta_test = pois_KL( X_test, y_test, SLOPE_final_beta, theta = theta_test)
        
#         eval LASSO
        LASSO_nll = -nb_ll( X_test, y_test, LASSO_final_beta)
        LASSO_KL = nb_KL( X_test, y_test, LASSO_final_beta, theta = None)
        LASSO_KL_theta_train = nb_KL( X_train, y_train, LASSO_final_beta, theta = theta_train)
        LASSO_KL_theta_test = nb_KL( X_test, y_test, LASSO_final_beta, theta = theta_test)
    
    
    model_scores_dict['SLOPE']['nll'] += SLOPE_nll
    model_scores_dict['SLOPE']['KL'] += SLOPE_KL
    model_scores_dict['SLOPE']['KL_theta_train'] += SLOPE_KL_theta_train
    model_scores_dict['SLOPE']['KL_theta_test'] += SLOPE_KL_theta_test
    model_scores_dict['SLOPE']['size'] += SLOPE_final_beta_size
    
    model_scores_dict['LASSO']['nll'] += LASSO_nll
    model_scores_dict['LASSO']['KL'] += LASSO_KL
    model_scores_dict['LASSO']['KL_theta_train'] += LASSO_KL_theta_train
    model_scores_dict['LASSO']['KL_theta_test'] += LASSO_KL_theta_test
    model_scores_dict['LASSO']['size'] += LASSO_final_beta_size
    
    print('Finished all')
    
    return model_scores_dict

def train_test_allocator(X, y, theta, n_test = 100):
    X_inds = [*range(X.shape[0])]
    test_sample = np.random.choice(X_inds, 100,replace = False)
    train_sample = [i for i in X_inds if i not in test_sample]
    train_set = (X[train_sample], y[train_sample], theta[train_sample])
    test_set = (X[test_sample], y[test_sample], theta[test_sample])
    return train_set, test_set

def matrix_simulator(d, d0
                     , rho
                     , beta_set = [0.5, -0.5, 0.6, -0.6]
                     , n = 300
                     , sim_num = 100):
    
    means = np.zeros(d)
    if rho == 0:
        conv_mat = np.diagflat(np.ones(d))
    else:
        conv_mat = cov_creator(d, rho)
    X = multivariate_normal(means, conv_mat, size = n*sim_num)
    X = X/norm(X, axis = 0)
    
    beta = beta_creator(d, d0, beta_set)
    theta = np.exp(np.dot(X,beta))
    y = poisson.rvs(theta) 
    return X, y, theta
    
def cov_creator(d, rho):
    arr = np.zeros(d)
    for i in range(d):
        arr[i] = rho**((i+1)-1)
    cov = np.zeros((d,d))
    for i in range(d):
        cov[i,:] = np.concatenate([arr[:i+1][::-1],arr[1:d-i]])
    return cov

def beta_creator(d, d0, beta_set = [0.5, -0.5, 0.6, -0.6]):
    b_d0 = np.random.choice(beta_set, d)
    zero_inds = np.random.choice([*range(d)],d-d0,replace = False)
    b_d0[zero_inds] = 0
    return b_d0

    
# def mp_apply():
# for d in simulation_settings:
#     # Simulate 100 matrices of 300 X d for each d_0
#     # And generate the dpendent variable
#     np.

## Simulation Run

In [ ]:
# import os
# pool = multiprocessing.Pool(os.cpu_count())
# print(pool)
# if __name__ == '__main__':
# # output = process_pool.starmap(f_sum, data)
# # results = {}
# # # for d in simulation_settings:
#     d = 20
#     for di in simulation_settings[d][:1]:
#         print('running d0: {}'.format(di))
#         for rho in [0, 0.5, 0.8][:1]:
#             print('running rho: {}'.format(rho))
#             data = matrix_simulator(d, di, rho = rho, sim_num = 8)
#     #             print(data)
#             X_split = np.vsplit(data[0], 8)
#             y_split = np.split(data[1], 8)
#             theta_split = np.split(data[2], 8)
#             map_args = [(X,y,theta, 'poisson') for X,y,theta in zip(X_split,y_split,theta_split)]
#     #         print(map_args[0])
# #             output = pool.starmap(runner, map_args)
#             output = pool.map(runner, map_args)
#             print("Finished Pooling")
        
# #             print(zip(X_split,y_split,theta_split))
# #         result_forward = pool.starmap()
# #     print(d, simulation_settings[d])
    

<multiprocessing.pool.Pool state=RUN pool_size=8>
running d0: 1
running rho: 0


### Numpy

In [5]:
import pickle
with open('sim_dict_0', 'rb') as f:
    results = pickle.load(f)

In [6]:
results.keys()

dict_keys([(20, 1, 0), (20, 2, 0), (20, 3, 0), (20, 4, 0), (20, 5, 0), (20, 6, 0), (20, 7, 0), (20, 10, 0), (20, 14, 0), (20, 18, 0), (100, 5, 0), (100, 10, 0), (100, 15, 0), (100, 20, 0), (100, 25, 0), (100, 30, 0), (100, 35, 0), (100, 50, 0), (100, 70, 0), (100, 90, 0)])

In [ ]:
import os
import multiprocessing
import itertools
import pickle
import numpy as np
from numpy.linalg import norm
from numpy.random import multivariate_normal, poisson
from numpy.linalg import inv
from numpy.linalg import multi_dot
from simulation_runner import runner as sim_runner
from simulation_runner import matrix_simulator



pool = multiprocessing.Pool(os.cpu_count())
# print(pool)

if __name__ == '__main__':
# output = process_pool.starmap(f_sum, data)
#     results = {}
    with open('sim_dict_0', 'rb') as f:
        results = pickle.load(f)
    for rho in [0, 0.5, 0.8]:
        for d in simulation_settings:
    #     d = 20
            print('running d: {}'.format(d))
            for di in simulation_settings[d]:
                if (d,di,rho) in results:
                    print('{} In dictionary'.format((d,di,rho)))
                    continue
                print('running d0: {}'.format(di))
#                 for rho in [0, 0.5, 0.8]:
                print('running rho: {}'.format(rho))
                data = matrix_simulator(d, di, rho = rho)
        #             print(data)
                X_split = np.vsplit(data[0], 100)
                y_split = np.split(data[1], 100)
                theta_split = np.split(data[2], 100)
                map_args = [(X,y,theta, 'poisson') for X,y,theta in zip(X_split,y_split,theta_split)]
        #         print(map_args[0])
                output = pool.starmap(sim_runner, map_args)
                results[(d,di,rho)] = output
                print('finished simulations of: {}'.format(d,di,rho))
                with open('sim_dict_{}'.format(rho), 'wb') as fp:
                    pickle.dump(results, fp)
    #             output = pool.map(sim_runner, map_args)
        print("Finished Pooling rho: {}".format(rho))
            
        
#             print(zip(X_split,y_split,theta_split))
#         result_forward = pool.starmap()
#     print(d, simulation_settings[d])
    


running d: 20
(20, 1, 0) In dictionary
(20, 2, 0) In dictionary
(20, 3, 0) In dictionary
(20, 4, 0) In dictionary
(20, 5, 0) In dictionary
(20, 6, 0) In dictionary
(20, 7, 0) In dictionary
(20, 10, 0) In dictionary
(20, 14, 0) In dictionary
(20, 18, 0) In dictionary
running d: 100
(100, 5, 0) In dictionary
(100, 10, 0) In dictionary
(100, 15, 0) In dictionary
(100, 20, 0) In dictionary
(100, 25, 0) In dictionary
(100, 30, 0) In dictionary
(100, 35, 0) In dictionary
(100, 50, 0) In dictionary
(100, 70, 0) In dictionary
(100, 90, 0) In dictionary
running d: 200
running d0: 10
running rho: 0


In [12]:
results

{(20,
  1,
  0): [{'FWD': {'nll': 102.10509670783853,
    'KL': 84550.20936529551,
    'KL_theta_train': 107450.24894538826,
    'KL_theta_test': 26587.98492985053,
    'size': 2},
   'LASSO': {'nll': 100.0,
    'KL': 84154.04825736102,
    'KL_theta_train': 108359.25424959138,
    'KL_theta_test': 27042.39315600711,
    'size': 0},
   'SLOPE': {'nll': 100.0,
    'KL': 84154.04825736102,
    'KL_theta_train': 108359.25424959138,
    'KL_theta_test': 27042.39315600711,
    'size': 0}}, {'FWD': {'nll': 99.91478243342443,
    'KL': 113212.54035226884,
    'KL_theta_train': 108483.52054739305,
    'KL_theta_test': 27015.611278017594,
    'size': 2},
   'LASSO': {'nll': 100.0,
    'KL': 113284.0597263231,
    'KL_theta_train': 108418.15895832314,
    'KL_theta_test': 27013.426456544214,
    'size': 0},
   'SLOPE': {'nll': 100.0,
    'KL': 113284.0597263231,
    'KL_theta_train': 108418.15895832314,
    'KL_theta_test': 27013.426456544214,
    'size': 0}}, {'FWD': {'nll': 100.34279913889554,

### Cupy

In [ ]:
import cupy as cp
import numpy as np
from cupy.linalg import norm, inv
# , dot
from cupy.random import multivariate_normal
from scipy.stats import poisson
from simulation_runner_cupy import runner as sim_runner
from simulation_runner_cupy import matrix_simulator 
import os
import multiprocessing
import itertools
import pickle

pool = multiprocessing.Pool(os.cpu_count())
# print(pool)

if __name__ == '__main__':
# output = process_pool.starmap(f_sum, data)
    results = {}
    for rho in [0, 0.5, 0.8]:
#         for d in simulation_settings:
        for d in [20]:
    #     d = 20
            print('running d: {}'.format(d))
            for di in simulation_settings[d]:
                print('running d0: {}'.format(di))
#                 for rho in [0, 0.5, 0.8]:
                print('running rho: {}'.format(rho))
                data = matrix_simulator(d, di, rho = rho)
        #             print(data)
                X_split = cp.asarray(np.vsplit(data[0], 100))
                y_split = cp.asarray(np.split(data[1], 100))
                theta_split = cp.asarray(np.split(data[2], 100))
                map_args = [(X,y,theta, 'poisson') for X,y,theta in zip(X_split,y_split,theta_split)]
#                 map_args = [{'X':X
#                              ,'y':y
#                              ,'theta':theta
#                              , 'reg_type':'poisson'} for X,y,theta in zip(X_split,y_split,theta_split)]
#                 print(map_args[0])
                output = pool.starmap(sim_runner, map_args)
#                 output = [sim_runner(**args) for args in map_args]
                results[(d,di,rho)] = output
                print('finished simulations of: {}'.format(d,di,rho))
                with open('sim_dict_{()}'.format(rho), 'wb') as fp:
                    pickle.dump(results, fp)
    #             output = pool.map(sim_runner, map_args)
        print("Finished Pooling rho: {}".format(rho))
            
        
#             print(zip(X_split,y_split,theta_split))
#         result_forward = pool.starmap()
#     print(d, simulation_settings[d])
    


running d: 20
running d0: 1
running rho: 0


In [13]:
# results[(d, di, rho)] = [res for res in output]
# results
output

[{'FWD': {'nll': 99.35713806900293,
   'KL': 113327.65288423005,
   'KL_theta_train': 107336.41232029392,
   'KL_theta_test': 26939.28584059106,
   'size': 2},
  'LASSO': {'nll': 100.0,
   'KL': 113795.92917122482,
   'KL_theta_train': 108232.40193697,
   'KL_theta_test': 26951.849003674917,
   'size': 0},
  'SLOPE': {'nll': 100.18733718367497,
   'KL': 113982.65439084038,
   'KL_theta_train': 108356.133129757,
   'KL_theta_test': 26977.957944755915,
   'size': 1}},
 {'FWD': {'nll': 100.69177586178331,
   'KL': 115216.86434877384,
   'KL_theta_train': 106929.0714084784,
   'KL_theta_test': 27156.31033316313,
   'size': 2},
  'LASSO': {'nll': 100.08865434459278,
   'KL': 119405.70620147133,
   'KL_theta_train': 108014.48957653274,
   'KL_theta_test': 27081.4053534604,
   'size': 0},
  'SLOPE': {'nll': 100.0,
   'KL': 119068.93285354205,
   'KL_theta_train': 108069.17305281028,
   'KL_theta_test': 27014.03422566439,
   'size': 0}},
 {'FWD': {'nll': 101.05881387225647,
   'KL': 103489.249

In [ ]:
import os
os.cpu_count()

In [111]:
d = 20
d0 = 5
matrix_simulator(20, 5, 0)

(array([[-5.78391609e-03, -4.03495220e-03, -4.18974836e-03, ...,
          2.66413076e-03,  1.14657787e-03,  3.98668167e-03],
        [ 2.08829305e-03,  3.11895185e-03,  2.12775132e-03, ...,
         -1.07492684e-02,  2.94142482e-04, -2.96663166e-04],
        [-1.63103498e-03,  5.23481844e-03, -9.67172344e-03, ...,
          4.15401949e-03,  1.73961241e-04,  1.17674197e-03],
        ...,
        [ 2.53987048e-03,  1.80619388e-04, -7.26024073e-04, ...,
         -1.10541895e-02, -5.31152890e-03, -5.14544308e-03],
        [-1.46999783e-03,  2.38073217e-05,  5.94246903e-03, ...,
          1.32458038e-04, -1.77938026e-03,  4.90695259e-03],
        [-3.88378712e-03,  1.78927393e-03,  2.50552796e-04, ...,
          2.00161648e-03,  4.07938619e-03, -8.57777341e-04]]),
 array([2, 4, 2, ..., 0, 1, 0]),
 array([1.00306099, 1.00326202, 0.9991098 , ..., 1.0004237 , 0.99905366,
        0.99958739]))